# Cirq Conversion

We provide `CirqGate` and `cirq_circuit_to_cbloq` to support translating Cirq gates and circuits into bloqs and composite bloqs. `CirqGate` uses the cirq "calling convention" of one thru-register composted of a 1d array of qubits.

In [ ]:
import cirq

from cirq_qubitization.jupyter_tools import show_bloq
from cirq_qubitization.quantum_graph.cirq_conversion import CirqGate

cg = CirqGate(cirq.CNOT)
cg

In [ ]:
show_bloq(cg)

### `cirq_circuit_to_cbloq`

A Cirq circuit can be converted to a composite bloq by wrapping each operation with `CirqGate`. 

In [ ]:
from cirq_qubitization.quantum_graph.cirq_conversion import cirq_circuit_to_cbloq

qubits = cirq.LineQubit.range(4)
circuit = cirq.testing.random_circuit(qubits, n_moments=5, op_density=1.0, random_state=52)
cbloq = cirq_circuit_to_cbloq(circuit)

print(circuit)
show_bloq(cbloq)

### Unitaries

Both containers support numerical contraction to a dense unitary matrix. Cirq contracts operations into a large identity matrix "in order". Bloqs use `quimb` to find a good contraction ordering and perform the contraction. 

In [ ]:
import numpy as np

bloq_unitary = cbloq.tensor_contract()
cirq_unitary = circuit.unitary(qubits)
np.testing.assert_allclose(cirq_unitary, bloq_unitary, atol=1e-8)

### Back to `cirq.Circuit`

In [ ]:
# Note: a 1d `wireshape` bloq register is actually two-dimensional in cirq-world
# because of the implicit `bitsize` dimension (which must be explicit in cirq-world).
# CirqGate has registers of bitsize=1 and wireshape=(n,); hence the list transpose below.
circuit2 = cbloq.to_cirq_circuit(qubits=[[q] for q in qubits])
circuit2

In [ ]:
# We lose the moment structure during the roundtrip.
circuit == circuit2

In [ ]:
# But the left-aligned `circuit` is recovered.
cirq.Circuit(circuit.all_operations()) == circuit2